In [124]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [125]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)
driver.get('https://www.law.go.kr/LSW/main.html')
driver.find_element_by_name('query').clear()
driver.find_element_by_name('query').send_keys('화학물질관리법')
driver.find_element_by_xpath('//*[@id="inner"]/a').click()
driver.find_element_by_xpath('//*[@id="liBgcolor0"]/a').click()

In [126]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [127]:
soup

<html lang="ko" style="overflow: hidden auto; height: 100%; border: none; padding: 0px; margin: 0px;" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script src="/LSW/js/jquery/jquery.js" type="text/javascript"></script>
<script src="/LSW/js/common/jquery-custom.js" type="text/javascript"></script>
<script src="/LSW/js/jquery/jquery.loadmask.js" type="text/javascript"></script>
<script type="text/javascript">
    var webRoot = '/LSW';
</script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/LSW/images/ico_favorites.ico" rel="shortcut icon" type="text/css"/>
<link href="/LSW/css/lsw/common.css" rel="stylesheet" type="text/css"/>
<link href="/LSW/js/jquery/jquery.loadmask.css" rel="stylesheet" type="text/css"/>
<link href="/LSW/css/ext/ext-all.css" rel="stylesheet" type="text/css"/>
<link href="/LSW/css/lsw/xtheme-layer.css" rel="stylesheet" type="text/css"/>
<link href="/LSW/css/siteMap.

In [128]:
items = soup.find_all('div',attrs={'class':'lawcon'})
info = []
for i in items:
    idx = 1
    details = {}
    # 조 가 들어갈 temp list
    hang_detail = []
    ho_detail = []
    details["조"] = i.label.get_text().strip()
    for j in i.find_all('p'):
        if j.attrs['class'][0][5:7] == "de":
            if len(j.attrs['class'][0]) > 8:  
                if j.attrs['class'][0][8] == "_":
                    hang_detail.append(j.get_text().strip())
                else:
                    ho_detail.append(j.get_text().strip())
        if j.attrs['class'][0][5] in ["p","y"]:
            if j.attrs['class'][0][5] == "p":
                hang_detail.append(j.get_text().strip()[len(i.label.get_text().strip()):-1].strip())
            else:
                hang_detail.append(j.get_text())
    details["항"] = hang_detail
    details["호"] = ho_detail
    info.append(details)
print(info[2]["항"])

['① 이 법은 다음 각 호의 어느 하나에 해당하는 화학물질에는 적용하지 아니한다', '② 제1항제13호에 해당하는 화학물질에 대하여는 제1항에도 불구하고 제4조부터 제23조까지(제6조제3항제1호부터 제5호까지, 제13조, 제16조, 제19조 및 제20조는 제외한다), 제23조의2, 제23조의3, 제26조, 제39조부터 제50조까지(제49조제1항제7호ㆍ제8호 및 제50조제1항제6호는 제외한다), 제52조, 제54조부터 제64조까지(제54조제8호부터 제13호까지, 제58조제4호ㆍ제5호, 제59조제7호부터 제9호까지, 제59조제11호, 제61조제4호, 제64조제1항제5호부터 제9호까지 및 제64조제2항제4호는 제외한다)를 적용한다. \xa0<개정 2016. 12. 27., 2017. 11. 28., 2020. 3. 31.>', '③ 제1항제2호부터 제13호까지의 규정에 따른 화학물질의 관리 및 화학사고 대응에 관하여 관계 법률에 다른 특별한 규정이 있는 경우를 제외하고는 제1항에도 불구하고 이 법에 따른다.']


In [86]:
items = soup.find_all('div',attrs={'class':'lawcon'})
info = []
for i in items:
    hang_idx = 1
    jo_idx = 1
    details = {}
    hang_detail = {}
    ho_detail = {}
    details["조"] = i.label.get_text().strip()
    for j in i.find_all('p'):
        if j.attrs['class'][0][5] in ["p","y"]:
            details["{}항".format(hang_idx)] = j.get_text().strip()[len(i.label.get_text().strip()):-1].strip()
            hang_idx += 1
        elif j.attrs['class'][0][5:7] == "de":
            if (len(j.attrs['class'][0]) > 8) :
                if (j.attrs['class'][0][8] == "_" ):
                    details["{}항".format(hang_idx)] = j.get_text().strip()
                    hang_idx += 1
                else:
                    ho_idx= j.get_text().strip().split(".")[0]
                    details["{}항{}호".format(hang_idx-1,ho_idx)] = j.get_text().strip()
                    
    info.append(details)
    
print(info)

[{'조': '제1조(목적)', '1항': '이 법은 화학물질로 인한 국민건강 및 환경상의 위해(危害)를 예방하고 화학물질을 적절하게 관리하는 한편, 화학물질로 인하여 발생하는 사고에 신속히 대응함으로써 화학물질로부터 모든 국민의 생명과 재산 또는 환경을 보호하는 것을 목적으로 한다'}, {'조': '제2조(정의)', '1항': '이 법에서 사용하는 용어의 뜻은 다음과 같다. \xa0<개정 2020. 5. 26.', '1항1호': '1. “화학물질”이란 원소ㆍ화합물 및 그에 인위적인 반응을 일으켜 얻어진 물질과 자연 상태에서 존재하는 물질을 화학적으로 변형시키거나 추출 또는 정제한 것을 말한다.', '1항2호': '2. “유독물질”이란 유해성(有害性)이 있는 화학물질로서 대통령령으로 정하는 기준에 따라 환경부장관이 정하여 고시한 것을 말한다.', '1항3호': '3. “허가물질”이란 위해성(危害性)이 있다고 우려되는 화학물질로서 환경부장관의 허가를 받아 제조, 수입, 사용하도록 환경부장관이 관계 중앙행정기관의 장과의 협의와 「화학물질의 등록 및 평가 등에 관한 법률」 제7조에 따른 화학물질평가위원회의 심의를 거쳐 고시한 것을 말한다.', '1항4호': '4. “제한물질”이란 특정 용도로 사용되는 경우 위해성이 크다고 인정되는 화학물질로서 그 용도로의 제조, 수입, 판매, 보관ㆍ저장, 운반 또는 사용을 금지하기 위하여 환경부장관이 관계 중앙행정기관의 장과의 협의와 「화학물질의 등록 및 평가 등에 관한 법률」 제7조에 따른 화학물질평가위원회의 심의를 거쳐 고시한 것을 말한다.', '1항5호': '5. “금지물질”이란 위해성이 크다고 인정되는 화학물질로서 모든 용도로의 제조, 수입, 판매, 보관ㆍ저장, 운반 또는 사용을 금지하기 위하여 환경부장관이 관계 중앙행정기관의 장과의 협의와 「화학물질의 등록 및 평가 등에 관한 법률」 제7조에 따른 화학물질평가위원회의 심의를 거쳐 고시한 것을 말한다.', '1항6호': '6. “사고대비물질”이란 화학물질 중에서 급성독성(急性

In [123]:
df = pd.DataFrame(info)
df


,조,1항,1항1호,1항2호,1항3호,1항4호,1항5호,1항6호,1항7호,1항8호,...,1항7의2호,1항9의2호,5항3호,1항2의2호,1항3의2호,1항3의3호,1항3의4호,1항10의2호,1항4의2호,2항3의2호
0,제1조(목적),이 법은 화학물질로 인한 국민건강 및 환경상의 위해(危害)를 예방하고 화학물질을 적...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,제2조(정의),이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020. 5. 26.,1. “화학물질”이란 원소ㆍ화합물 및 그에 인위적인 반응을 일으켜 얻어진 물질과 자...,2. “유독물질”이란 유해성(有害性)이 있는 화학물질로서 대통령령으로 정하는 기준에...,3. “허가물질”이란 위해성(危害性)이 있다고 우려되는 화학물질로서 환경부장관의 허...,4. “제한물질”이란 특정 용도로 사용되는 경우 위해성이 크다고 인정되는 화학물질로...,"5. “금지물질”이란 위해성이 크다고 인정되는 화학물질로서 모든 용도로의 제조, 수...",6. “사고대비물질”이란 화학물질 중에서 급성독성(急性毒性)ㆍ폭발성 등이 강하여 화...,"7. “유해화학물질”이란 유독물질, 허가물질, 제한물질 또는 금지물질, 사고대비물질...",8. “유해화학물질 영업”이란 유해화학물질 중 허가물질 및 금지물질을 제외한 나머지...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,제3조(적용범위),① 이 법은 다음 각 호의 어느 하나에 해당하는 화학물질에는 적용하지 아니한다. ...,1. 「원자력안전법」 제2조제5호에 따른 방사성물질,2. 「약사법」 제2조제4호ㆍ제7호에 따른 의약품 및 의약외품,3. 「마약류 관리에 관한 법률」 제2조제1호에 따른 마약류,4. 「화장품법」 제2조제1호에 따른 화장품과 화장품에 사용하는 원료,5. 「농약관리법」 제2조제1호ㆍ제3호에 따른 농약과 원제(原劑),6. 「비료관리법」 제2조제1호에 따른 비료,"7. 「식품위생법」 제2조제1호ㆍ제2호ㆍ제4호ㆍ제5호에 따른 식품, 식품첨가물, 기...",8. 「사료관리법」 제2조제1호에 따른 사료,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,제4조(국가 및 지방자치단체의 책무),① 국가 및 지방자치단체는 화학물질의 유해성ㆍ위해성으로부터 국민건강과 환경에 미치는...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,제5조(화학물질 취급자의 책무),① 화학물질을 취급하는 자는 화학물질로 인한 국민건강상 또는 환경상의 위해가 발생하...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,제60조(벌칙),제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징역 또는 1억원 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,제61조(벌칙),다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3천만원 이하의 벌...,1. 제19조제5항에 따른 허가조건을 이행하지 하지 아니한 자,2. 제20조제2항에 따른 유독물질 수입신고를 하지 아니하거나 거짓으로 신고하고 수...,3. 제21조제1항 전단에 따른 제한물질의 수출승인을 받지 아니하거나 거짓으로 승인...,4. 제28조제5항 전단에 따른 유해화학물질 영업의 변경허가를 받지 아니하거나 거짓...,5. 제29조의3에 따른 유해화학물질에 해당하는 시험용ㆍ연구용ㆍ검사용 시약의 판매업...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3의2. 제23조제3항에 따른 변경된 화학사고예방관리계획서를 제출하지 아니하거나 거...,3의3. 제23조제7항에 따른 화학사고예방관리계획서를 수정ㆍ보완하여 제출하지 아니한 자,3의4. 제23조의2제3항에 따른 시정명령 등에 따르지 아니한 자,NaN,NaN,NaN
73,제62조(벌칙),다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 500만원 이하의...,1. 제18조제1항 단서를 위반하여 금지물질의 제조ㆍ수입ㆍ판매 허가를 받지 아니하거...,2. 제18조제2항에 따른 변경허가를 받지 아니하거나 거짓으로 변경허가를 받고 금지...,3. 제21조제1항 후단에 따른 제한물질ㆍ금지물질의 수출에 대한 변경승인을 받지 아...,4. 제28조의2를 위반하여 구매자의 실명ㆍ연령 확인 또는 본인 인증을 거치지 아니...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,제63조(양벌규정),"법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종업원이 그 법인 또는...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
items = soup.find_all('div',attrs={'class':'lawcon'})
info = []
jo = []
hang = []
ho = []
idx = []
for i in items:
    details = {}
    label = i.label.get_text().strip()
    
    
    if '조의' in label:
        num_jo = label.split('조')[0][1:] + '조의' + label.split('조')[1][1]
    else :
        num_jo = label.split('조')[0][1:] + '조'
    #print(num_jo)
    
    jo.append(num_jo)
#details['조문번호'] = jo    
    for j in i.find_all('p'):
        if j.attrs['class'][0][5:7] in ['p1']:
            details['조문내용'] = j.get_text().strip()[len(i.label.get_text().strip()):-1].strip()
        if j.attrs['class'][0][5:7] in ['p4'] or j.attrs['class'][0][5:9] in ['de2_']:
            details['항문내용'] = j.get_text().strip()
        if j.attrs['class'][0][5:9] in ['de2h']:
            details['호문내용'] = j.get_text().strip()

    info.append(details)
    
print(info)

[{'항문내용': '제1조(목적) 이 법은 화학물질로 인한 국민건강 및 환경상의 위해(危害)를 예방하고 화학물질을 적절하게 관리하는 한편, 화학물질로 인하여 발생하는 사고에 신속히 대응함으로써 화학물질로부터 모든 국민의 생명과 재산 또는 환경을 보호하는 것을 목적으로 한다.'}, {'조문내용': '이 법에서 사용하는 용어의 뜻은 다음과 같다. \xa0<개정 2020. 5. 26.', '호문내용': '13. “화학사고”란 시설의 교체 등 작업 시 작업자의 과실, 시설 결함ㆍ노후화, 자연재해, 운송사고 등으로 인하여 화학물질이 사람이나 환경에 유출ㆍ누출되어 발생하는 모든 상황을 말한다.'}, {'조문내용': '① 이 법은 다음 각 호의 어느 하나에 해당하는 화학물질에는 적용하지 아니한다', '호문내용': '13. 「고압가스 안전관리법」에 따른 독성 가스', '항문내용': '③ 제1항제2호부터 제13호까지의 규정에 따른 화학물질의 관리 및 화학사고 대응에 관하여 관계 법률에 다른 특별한 규정이 있는 경우를 제외하고는 제1항에도 불구하고 이 법에 따른다.'}, {'조문내용': '① 국가 및 지방자치단체는 화학물질의 유해성ㆍ위해성으로부터 국민건강과 환경에 미치는 영향을 늘 파악하고, 국민건강이나 환경상의 위해를 예방하기 위하여 필요한 시책을 수립ㆍ시행하여야 한다', '항문내용': '④ 환경부장관은 중소기업의 화학물질 안전관리를 위하여 관계 중앙행정기관의 장과 협의를 거쳐 다음 각 호의 사항에 대한 행정적ㆍ기술적ㆍ재정적 지원방안을 마련하여 시행할 수 있다. \xa0<신설 2016. 1. 27., 2020. 3. 31.>', '호문내용': '3. 그 밖에 중소기업의 화학물질 안전관리를 위하여 대통령령으로 정하는 사항'}, {'항문내용': '② 화학물질을 취급하는 자는 해당 화학물질의 안전한 관리에 관한 책임을 진다.'}, {'항문내용': '⑤ 관계 중앙행정기관의 장 및 지방자치단체의 장은 기본계획에 따라 소관 사항에 속하는 시책을 수립ㆍ시행하여야 한다.', '호문내용': 

In [131]:
df = pd.DataFrame(info)
df.index = jo
df

,항문내용,조문내용,호문내용
1조,제1조(목적) 이 법은 화학물질로 인한 국민건강 및 환경상의 위해(危害)를 예방하고...,NaN,NaN
2조,NaN,이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020. 5. 26.,"13. “화학사고”란 시설의 교체 등 작업 시 작업자의 과실, 시설 결함ㆍ노후화, ..."
3조,③ 제1항제2호부터 제13호까지의 규정에 따른 화학물질의 관리 및 화학사고 대응에 ...,① 이 법은 다음 각 호의 어느 하나에 해당하는 화학물질에는 적용하지 아니한다,13. 「고압가스 안전관리법」에 따른 독성 가스
4조,④ 환경부장관은 중소기업의 화학물질 안전관리를 위하여 관계 중앙행정기관의 장과 협의...,① 국가 및 지방자치단체는 화학물질의 유해성ㆍ위해성으로부터 국민건강과 환경에 미치는...,3. 그 밖에 중소기업의 화학물질 안전관리를 위하여 대통령령으로 정하는 사항
5조,② 화학물질을 취급하는 자는 해당 화학물질의 안전한 관리에 관한 책임을 진다.,NaN,NaN
...,...,...,...
60조,제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...,NaN,NaN
61조,NaN,다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3천만원 이하의 벌...,5. 제29조의3에 따른 유해화학물질에 해당하는 시험용ㆍ연구용ㆍ검사용 시약의 판매업...
62조,NaN,다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 500만원 이하의...,4. 제28조의2를 위반하여 구매자의 실명ㆍ연령 확인 또는 본인 인증을 거치지 아니...
63조,"제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종...",NaN,NaN


In [132]:
jo

['1조',
 '2조',
 '3조',
 '4조',
 '5조',
 '6조',
 '7조',
 '7조의2',
 '8조',
 '9조',
 '10조',
 '11조',
 '11조의2',
 '12조',
 '13조',
 '14조',
 '15조',
 '16조',
 '17조',
 '18조',
 '19조',
 '20조',
 '21조',
 '22조',
 '23조',
 '23조의2',
 '23조의3',
 '23조의4',
 '24조',
 '25조',
 '26조',
 '27조',
 '28조',
 '28조의2',
 '29조',
 '29조의2',
 '29조의3',
 '30조',
 '31조',
 '32조',
 '33조',
 '34조',
 '34조의2',
 '35조',
 '36조',
 '37조',
 '38조',
 '39조',
 '40조',
 '41조',
 '41조의2',
 '42조',
 '43조',
 '44조',
 '44조의2',
 '45조',
 '46조',
 '47조',
 '48조',
 '49조',
 '50조',
 '51조',
 '52조',
 '53조',
 '54조',
 '55조',
 '56조',
 '57조',
 '58조',
 '59조',
 '60조',
 '61조',
 '62조',
 '63조',
 '64조']